In [ ]:
# Cell1:  Shared Cache Bootstrap
import os, pathlib, torch
import sys
from datetime import datetime

# Shared cache configuration (複製到每本 notebook)
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "../ai_warehouse/cache")

for k, v in {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)
print("[Cache]", AI_CACHE_ROOT, "| GPU:", torch.cuda.is_available())

In [ ]:
# Cell 2: Dependencies & Imports
import json, time, asyncio, traceback
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Any, Union
from enum import Enum
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Cell 3: Blackboard State Management
class Blackboard(dict):
    """
    Shared state management for multi-agent collaboration.
    Stores research results, plans, drafts, and review feedback.
    """

    def __init__(self):
        super().__init__()
        self.update(
            {
                "query": "",
                "research_results": [],
                "plan": [],
                "draft": "",
                "review_feedback": [],
                "final_output": "",
                "metadata": {"start_time": time.time(), "iterations": 0, "errors": []},
            }
        )

    def log_error(self, agent: str, error: str):
        """Log errors for debugging"""
        self["metadata"]["errors"].append(
            {"agent": agent, "error": error, "timestamp": time.time()}
        )

    def get_context(self) -> str:
        """Get formatted context for agents"""
        context = f"Query: {self['query']}\n"
        if self["research_results"]:
            context += f"Research: {len(self['research_results'])} findings\n"
        if self["plan"]:
            context += f"Plan: {len(self['plan'])} sections\n"
        if self["draft"]:
            context += f"Draft: {len(self['draft'])} chars\n"
        return context

In [ ]:
# Cell 4: Agent Role Definitions
class AgentRole(Enum):
    RESEARCHER = "researcher"
    PLANNER = "planner"
    WRITER = "writer"
    REVIEWER = "reviewer"


@dataclass
class AgentConfig:
    """Configuration for each agent role"""

    max_tokens: int = 512
    temperature: float = 0.7
    timeout: int = 60
    max_retries: int = 2


# Default configs for each role
AGENT_CONFIGS = {
    AgentRole.RESEARCHER: AgentConfig(max_tokens=1024, temperature=0.3),
    AgentRole.PLANNER: AgentConfig(max_tokens=512, temperature=0.5),
    AgentRole.WRITER: AgentConfig(max_tokens=2048, temperature=0.7),
    AgentRole.REVIEWER: AgentConfig(max_tokens=512, temperature=0.2),
}

In [ ]:
# Cell 5: Base Agent Class
class BaseAgent:
    """Base class for all agents with common functionality"""

    def __init__(self, role: AgentRole, llm_adapter, config: AgentConfig = None):
        self.role = role
        self.llm = llm_adapter
        self.config = config or AGENT_CONFIGS[role]
        self.logger = logging.getLogger(f"Agent.{role.value}")

    def _create_messages(self, system_prompt: str, user_prompt: str) -> List[Dict]:
        """Create messages format for LLM"""
        return [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]

    def _call_llm(self, messages: List[Dict]) -> str:
        """Safe LLM call with error handling"""
        try:
            response = self.llm.generate(
                messages=messages,
                max_new_tokens=self.config.max_tokens,
                temperature=self.config.temperature,
            )
            # Extract only the assistant's response
            if "assistant:" in response:
                return response.split("assistant:")[-1].strip()
            return response.strip()
        except Exception as e:
            self.logger.error(f"LLM call failed: {e}")
            return f"[ERROR] {str(e)}"

    def execute(self, blackboard: Blackboard) -> bool:
        """Execute agent's task - to be overridden by subclasses"""
        raise NotImplementedError

In [ ]:
# Cell 6: Individual Agent Implementations
class ResearcherAgent(BaseAgent):
    """Researcher: Gathers information using RAG and web search"""

    def __init__(self, llm_adapter, rag_retriever=None):
        super().__init__(AgentRole.RESEARCHER, llm_adapter)
        self.rag = rag_retriever

    def execute(self, blackboard: Blackboard) -> bool:
        """Research the query and populate research_results"""
        query = blackboard["query"]

        system_prompt = """You are a research assistant. Your job is to gather relevant information and provide key findings with citations where possible.
Focus on factual information and cite sources. Keep each finding concise but informative."""

        # Use RAG if available
        context = ""
        if self.rag:
            try:
                # Simplified RAG call (assumes retrieve method exists)
                rag_results = self.rag.search(query, k=5)
                context = "\n".join(
                    [f"[{i+1}] {item[0]}" for i, item in enumerate(rag_results)]
                )
                context = f"Available knowledge:\n{context}\n\n"
            except Exception as e:
                self.logger.warning(f"RAG search failed: {e}")

        user_prompt = f"""{context}Research query: {query}

Provide 3-5 key research findings. Format each as:
- Finding: [brief description]
- Source: [if available]
- Relevance: [why important for this query]"""

        messages = self._create_messages(system_prompt, user_prompt)
        response = self._call_llm(messages)

        if not response.startswith("[ERROR]"):
            blackboard["research_results"].append(
                {"agent": "researcher", "content": response, "timestamp": time.time()}
            )
            return True
        else:
            blackboard.log_error("researcher", response)
            return False


class PlannerAgent(BaseAgent):
    """Planner: Creates structured outline based on research"""

    def execute(self, blackboard: Blackboard) -> bool:
        """Create a structured plan based on research results"""
        query = blackboard["query"]
        research = blackboard["research_results"]

        system_prompt = """You are a planning assistant. Create a clear, structured outline for addressing the user's query based on research findings.
Focus on logical flow and completeness. Keep each section concise but comprehensive."""

        research_summary = "\n".join(
            [
                f"Research {i+1}: {r['content'][:200]}..."
                for i, r in enumerate(research[-3:])  # Last 3 research results
            ]
        )

        user_prompt = f"""Query: {query}

Available research:
{research_summary}

Create a structured outline with 3-5 main sections. Format as:
1. Section Title: Brief description
2. Section Title: Brief description
...

Each section should be actionable and directly address the query."""

        messages = self._create_messages(system_prompt, user_prompt)
        response = self._call_llm(messages)

        if not response.startswith("[ERROR]"):
            # Parse plan into sections
            sections = []
            for line in response.split("\n"):
                if line.strip() and (
                    line.strip()[0].isdigit() or line.strip().startswith("-")
                ):
                    sections.append(line.strip())

            blackboard["plan"] = sections
            return True
        else:
            blackboard.log_error("planner", response)
            return False


class WriterAgent(BaseAgent):
    """Writer: Produces content based on plan and research"""

    def execute(self, blackboard: Blackboard) -> bool:
        """Write content following the plan and incorporating research"""
        query = blackboard["query"]
        plan = blackboard["plan"]
        research = blackboard["research_results"]

        system_prompt = """You are a skilled writer. Create comprehensive, well-structured content that follows the provided outline and incorporates research findings.
Write in Traditional Chinese with clear explanations. Include citations where appropriate."""

        plan_text = "\n".join(plan)
        research_text = "\n\n".join(
            [r["content"] for r in research[-2:]]
        )  # Last 2 research items

        user_prompt = f"""Query: {query}

Outline to follow:
{plan_text}

Research to incorporate:
{research_text}

Write a comprehensive response (300-800 words) that:
1. Directly answers the query
2. Follows the outline structure
3. Incorporates research findings with citations
4. Uses Traditional Chinese
5. Is clear and well-organized"""

        messages = self._create_messages(system_prompt, user_prompt)
        response = self._call_llm(messages)

        if not response.startswith("[ERROR]"):
            blackboard["draft"] = response
            return True
        else:
            blackboard.log_error("writer", response)
            return False


class ReviewerAgent(BaseAgent):
    """Reviewer: Evaluates content quality and provides feedback"""

    def execute(self, blackboard: Blackboard) -> bool:
        """Review the draft and provide feedback"""
        query = blackboard["query"]
        draft = blackboard["draft"]

        if not draft:
            blackboard.log_error("reviewer", "No draft available to review")
            return False

        system_prompt = """You are a quality reviewer. Evaluate content for accuracy, completeness, clarity, and alignment with the original query.
Provide constructive feedback and suggest improvements. Be specific and actionable."""

        user_prompt = f"""Original query: {query}

Draft to review:
{draft}

Evaluate this draft on:
1. Accuracy - Are the facts correct?
2. Completeness - Does it fully address the query?
3. Clarity - Is it well-organized and easy to understand?
4. Citations - Are sources properly referenced?

Provide specific feedback and a quality score (1-10). If score ≥7, approve for final output."""

        messages = self._create_messages(system_prompt, user_prompt)
        response = self._call_llm(messages)

        if not response.startswith("[ERROR]"):
            blackboard["review_feedback"].append(
                {"content": response, "timestamp": time.time()}
            )

            # Simple approval logic - look for score ≥7 or positive keywords
            is_approved = any(
                keyword in response.lower()
                for keyword in [
                    "score: 7",
                    "score: 8",
                    "score: 9",
                    "score: 10",
                    "approve",
                    "good quality",
                    "well done",
                ]
            )

            if is_approved:
                blackboard["final_output"] = draft

            return True
        else:
            blackboard.log_error("reviewer", response)
            return False

In [ ]:
# Cell 7: Core Orchestrator Class
class Orchestrator:
    """
    Main orchestrator that coordinates the 4-agent workflow.
    Manages execution order, retries, and error handling.
    """

    def __init__(self, llm_adapter, rag_retriever=None, max_iterations=3, timeout=300):
        self.llm = llm_adapter
        self.rag = rag_retriever
        self.max_iterations = max_iterations
        self.timeout = timeout

        # Initialize agents
        self.agents = {
            AgentRole.RESEARCHER: ResearcherAgent(llm_adapter, rag_retriever),
            AgentRole.PLANNER: PlannerAgent(llm_adapter),
            AgentRole.WRITER: WriterAgent(llm_adapter),
            AgentRole.REVIEWER: ReviewerAgent(llm_adapter),
        }

        self.logger = logging.getLogger("Orchestrator")

    def execute_workflow(self, query: str) -> Dict[str, Any]:
        """
        Execute the complete 4-agent workflow.
        Returns final result with metadata.
        """
        start_time = time.time()
        blackboard = Blackboard()
        blackboard["query"] = query

        self.logger.info(f"Starting workflow for query: {query[:100]}...")

        try:
            for iteration in range(self.max_iterations):
                blackboard["metadata"]["iterations"] = iteration + 1

                # Check timeout
                if time.time() - start_time > self.timeout:
                    self.logger.error("Workflow timeout reached")
                    break

                # Execute agent sequence
                workflow_success = self._execute_agent_sequence(blackboard)

                if workflow_success and blackboard["final_output"]:
                    self.logger.info(
                        f"Workflow completed successfully in {iteration + 1} iterations"
                    )
                    break
                elif iteration == self.max_iterations - 1:
                    self.logger.warning("Max iterations reached without approval")
                    # Use draft as final output if available
                    if blackboard["draft"]:
                        blackboard["final_output"] = blackboard["draft"]

        except Exception as e:
            self.logger.error(f"Workflow failed: {e}")
            blackboard.log_error("orchestrator", str(e))

        # Compile final result
        result = {
            "query": query,
            "final_output": blackboard.get("final_output", "未能完成任務"),
            "research_count": len(blackboard["research_results"]),
            "plan_sections": len(blackboard["plan"]),
            "review_feedback": blackboard["review_feedback"],
            "iterations": blackboard["metadata"]["iterations"],
            "duration": time.time() - start_time,
            "errors": blackboard["metadata"]["errors"],
        }

        return result

    def _execute_agent_sequence(self, blackboard: Blackboard) -> bool:
        """Execute the standard agent sequence: Research → Plan → Write → Review"""

        sequence = [
            (AgentRole.RESEARCHER, "研究階段"),
            (AgentRole.PLANNER, "規劃階段"),
            (AgentRole.WRITER, "寫作階段"),
            (AgentRole.REVIEWER, "審核階段"),
        ]

        for role, stage_name in sequence:
            self.logger.info(f"Executing {stage_name} ({role.value})")

            agent = self.agents[role]
            success = False

            # Retry logic for each agent
            for attempt in range(agent.config.max_retries + 1):
                try:
                    success = agent.execute(blackboard)
                    if success:
                        break
                    else:
                        self.logger.warning(
                            f"{role.value} attempt {attempt + 1} failed"
                        )
                except Exception as e:
                    self.logger.error(f"{role.value} attempt {attempt + 1} error: {e}")
                    blackboard.log_error(role.value, str(e))

            if not success:
                self.logger.error(f"{stage_name} failed after all retries")
                return False

        return True

In [ ]:
# Cell 8: Integration Setup (Mock LLM for testing)
class MockLLMAdapter:
    """Mock LLM adapter for testing when real models aren't available"""

    def generate(self, messages, max_new_tokens=256, temperature=0.7):
        # Extract the last user message for context
        user_content = ""
        for msg in reversed(messages):
            if msg["role"] == "user":
                user_content = msg["content"]
                break

        # Generate mock responses based on content
        if "research" in user_content.lower():
            return """- Finding: 這是一個關於機器學習的重要概念
- Source: 學術文獻
- Relevance: 直接回答用戶問題

- Finding: RAG 技術提高了回答準確性
- Source: 最新研究
- Relevance: 與查詢高度相關"""

        elif "outline" in user_content.lower() or "plan" in user_content.lower():
            return """1. 基本概念介紹：定義和背景
2. 技術原理：詳細解釋工作機制
3. 實際應用：案例分析和使用場景
4. 總結建議：關鍵要點和最佳實踐"""

        elif "write" in user_content.lower() or "comprehensive" in user_content.lower():
            return """基於研究結果，我來詳細回答您的問題。

## 基本概念介紹
這個主題涉及多個重要概念，需要從基礎開始理解。

## 技術原理
核心機制包括幾個關鍵步驟，每個步驟都有其特定作用。

## 實際應用
在實際場景中，這些技術被廣泛應用於解決各種問題。

## 總結建議
總結來說，掌握這些概念對於理解整個領域非常重要。"""

        elif "review" in user_content.lower():
            return """Quality review:
1. Accuracy: 內容準確，基於可靠來源 - 8/10
2. Completeness: 涵蓋了主要方面 - 7/10
3. Clarity: 結構清晰，易於理解 - 8/10
4. Citations: 引用適當 - 7/10

Overall score: 8/10 - Approve for final output. 文章品質良好，建議發布。"""

        return "Mock response generated for testing."

In [ ]:
# Cell 9: Smoke Test
def run_smoke_test():
    """Test the orchestrator with a simple query"""
    print("=== 四角色協調器煙霧測試 ===")

    # Use mock LLM for testing
    mock_llm = MockLLMAdapter()

    # Initialize orchestrator
    orchestrator = Orchestrator(
        llm_adapter=mock_llm,
        rag_retriever=None,  # No RAG for smoke test
        max_iterations=2,
        timeout=120,
    )

    # Test query
    test_query = "什麼是 RAG（檢索增強生成）技術？"

    print(f"測試查詢: {test_query}")
    print("-" * 50)

    # Execute workflow
    result = orchestrator.execute_workflow(test_query)

    # Display results
    print(f"✅ 完成狀態: {'成功' if result['final_output'] else '失敗'}")
    print(f"📊 統計數據:")
    print(f"  - 研究結果: {result['research_count']} 條")
    print(f"  - 規劃章節: {result['plan_sections']} 個")
    print(f"  - 迭代次數: {result['iterations']}")
    print(f"  - 執行時間: {result['duration']:.2f} 秒")
    print(f"  - 錯誤數量: {len(result['errors'])}")

    if result["errors"]:
        print("⚠️  錯誤詳情:")
        for error in result["errors"]:
            print(f"  - {error['agent']}: {error['error'][:100]}...")

    print(f"\n📝 最終輸出:")
    print(
        result["final_output"][:300] + "..."
        if len(result["final_output"]) > 300
        else result["final_output"]
    )

    print(f"\n💭 審核反饋:")
    for i, feedback in enumerate(result["review_feedback"]):
        print(f"審核 {i+1}: {feedback['content'][:150]}...")

    return result


# Run the smoke test
smoke_result = run_smoke_test()

In [ ]:
# Cell 10: Error Handling & Circuit Breaker Patterns
class CircuitBreaker:
    """Simple circuit breaker for agent failure handling"""

    def __init__(self, failure_threshold=3, timeout=60):
        self.failure_threshold = failure_threshold
        self.timeout = timeout
        self.failure_count = 0
        self.last_failure_time = None
        self.state = "CLOSED"  # CLOSED, OPEN, HALF_OPEN

    def call(self, func, *args, **kwargs):
        """Execute function with circuit breaker protection"""
        if self.state == "OPEN":
            if time.time() - self.last_failure_time > self.timeout:
                self.state = "HALF_OPEN"
            else:
                raise Exception("Circuit breaker is OPEN")

        try:
            result = func(*args, **kwargs)
            if self.state == "HALF_OPEN":
                self.state = "CLOSED"
                self.failure_count = 0
            return result
        except Exception as e:
            self.failure_count += 1
            self.last_failure_time = time.time()

            if self.failure_count >= self.failure_threshold:
                self.state = "OPEN"

            raise e


# Usage example
def test_circuit_breaker():
    """Test circuit breaker functionality"""
    print("=== 斷路器測試 ===")

    breaker = CircuitBreaker(failure_threshold=2, timeout=5)

    def failing_function():
        raise Exception("模擬失敗")

    def working_function():
        return "成功執行"

    # Test failures
    for i in range(3):
        try:
            result = breaker.call(failing_function)
        except Exception as e:
            print(f"嘗試 {i+1}: {e} (狀態: {breaker.state})")

    # Test recovery after timeout
    time.sleep(6)  # Wait for timeout
    try:
        result = breaker.call(working_function)
        print(f"恢復後: {result} (狀態: {breaker.state})")
    except Exception as e:
        print(f"恢復失敗: {e}")


test_circuit_breaker()

print("\n🎯 關鍵參數說明:")
print("- max_iterations: 最大協作輪數 (預設 3)")
print("- timeout: 總時間限制秒數 (預設 300)")
print("- max_retries: 每個代理最大重試次數 (預設 2)")
print("- temperature: 各角色創造性設定 (研究員 0.3, 寫作者 0.7)")

print("\n📋 使用時機:")
print("- 需要多步驟推理和協作的複雜任務")
print("- 要求高品質輸出且有驗證機制的場景")
print("- 整合研究、規劃、寫作、審核的完整工作流")
print("- 需要容錯和重試機制的生產環境")